In [6]:
import os
import pandas as pd
import torch

from learning_models.sidarthe import Sidarthe
from populations import populations

from utils.data_utils import select_data
from utils.visualization_utils import generic_plot, Curve, format_xtick, generic_sub_plot, Plot

from torch_euler import Heun
import json

In [7]:
# load targets
df_file = os.path.join(os.getcwd(), "COVID-19", "dati-andamento-nazionale", "dpc-covid19-ita-andamento-nazionale.csv")
area = ["ITA"]
area_col_name = "stato"  # "Country/Region"

groupby_cols = ["data"]  # ["Date"]

d_col_name = "isolamento_domiciliare"
r_col_name = "ricoverati_con_sintomi"
t_col_name = "terapia_intensiva"
h_detected_col_name = "dimessi_guariti"
e_col_name = "deceduti"  # "Fatalities"

x_target, d_target = select_data(df_file, area, area_col_name, d_col_name, groupby_cols, file_sep=",")
_, y_target = select_data(df_file, area, area_col_name, "totale_positivi", groupby_cols, file_sep=",")
_, r_target = select_data(df_file, area, area_col_name, r_col_name, groupby_cols, file_sep=",")
_, t_target = select_data(df_file, area, area_col_name, t_col_name, groupby_cols, file_sep=",")
_, h_detected_target = select_data(df_file, area, area_col_name, h_detected_col_name, groupby_cols, file_sep=",")
_, e_target = select_data(df_file, area, area_col_name, e_col_name, groupby_cols, file_sep=",")

initial_len = len(y_target)
tmp_d, tmp_r, tmp_t, tmp_h, tmp_e = [], [], [], [], []
for i in range(initial_len):
    if y_target[i] > 0:
        tmp_d.append(d_target[i])
        tmp_r.append(r_target[i])
        tmp_t.append(t_target[i])
        tmp_h.append(h_detected_target[i])
        tmp_e.append(e_target[i])
d_target = tmp_d
r_target = tmp_r
t_target = tmp_t
h_detected_target = tmp_h
e_target = tmp_e

targets = {
    "d": d_target,
    "r": r_target,
    "t": t_target,
    "h_detected": h_detected_target,
    "e": e_target
}


In [8]:
# load references
references = {}
param_keys = ['alpha', 'beta', 'gamma', 'delta', 'epsilon', 'theta', 'xi', 'eta', 'mu', 'nu', 'tau', 'lambda', 'kappa', 'zeta', 'rho', 'sigma']
ref_df = pd.read_csv(os.path.join(os.getcwd(), "regioni", "sidarthe_results.csv"))
for key in 'sidarthe':
    references[key] = ref_df[key].tolist()

for key in ["r0", "h_detected"]:
    references[key] = ref_df[key].tolist()

for key in param_keys:
    references[key] = ref_df[key].tolist()

In [9]:
# load experiment values
exp_paths = os.path.join(os.getcwd(), "regioni_sidarthe")
exp_id = "52260e50-c7e4-45cb-b3b6-e11b1b51335a"
exp_path = os.path.join(exp_paths, exp_id)
exp_settings_path = os.path.join(exp_path, "settings.json")
exp_report_path = os.path.join(exp_path, "final.json")

with open(exp_settings_path) as settings_json:
    exp_settings = json.load(settings_json)

with open(exp_report_path) as report_json:
    exp_report = json.load(report_json)

In [24]:
# create trained model
population = populations["Italy"]
integrator = Heun
time_step = 1.

params = exp_report["params"]
train_size = exp_settings["train_size"]
val_size = exp_settings["val_len"]
dataset_size = len(x_target)

model_params = { 
    "d_weight": 1.,
    "r_weight": 1.,
    "t_weight": 1.,
    "h_weight": 1.,
    "e_weight": 1.,
    "der_1st_reg": 0.,
    "bound_reg": 0.,
    "verbose": False,
    "loss_type": "rmse",
    "references": references,
    "targets": targets,
    "train_size": train_size,
    "val_size": val_size
}
init_conditions_params = { "population": population }
initial_conditions = Sidarthe.compute_initial_conditions_from_targets(targets, init_conditions_params)

model = Sidarthe(params, population, initial_conditions, integrator, time_step, **model_params)

In [26]:
# compute inference
with torch.no_grad():
    t_start = 0
    t_end = train_size

    t_grid = torch.linspace(0, 100, int(100 / time_step) + 1)
    inferences = model.inference(t_grid)

In [91]:
# slice dataset
train_hat_slice = slice(t_start, int(train_size / time_step), int(1 / time_step))
val_hat_slice = slice(int(train_size / time_step), int(train_size + val_size / time_step),int(1 / time_step))
test_hat_slice = slice(int(train_size + val_size / time_step), int(dataset_size / time_step), int(1 / time_step))
dataset_hat_slice = slice(t_start, int(dataset_size / time_step), int(1 / time_step))

train_target_slice = slice(t_start, train_size, 1)
val_target_slice = slice(train_size, train_size + val_size, 1)
test_target_slice = slice(train_size + val_size, dataset_size, 1)
dataset_target_slice = slice(t_start, dataset_size, 1)

def slice_values(values, slice_):
    return {key: value[slice_] for key, value in values.items()}

hat_train = slice_values(inferences, train_hat_slice)
hat_val = slice_values(inferences, val_hat_slice)
hat_test = slice_values(inferences, test_hat_slice)
hat_dataset = slice_values(inferences, dataset_hat_slice)

target_train = slice_values(targets, train_target_slice)
target_val = slice_values(targets, val_target_slice)
target_test = slice_values(targets, test_target_slice)
target_dataset = slice_values(targets, dataset_target_slice)


references = { k: torch.tensor(v, dtype=model.dtype) for k,v in references.items() }

references_train = slice_values(references, train_target_slice)
references_val = slice_values(references, val_target_slice)
references_test = slice_values(references, test_target_slice)
references_dataset = slice_values(references, dataset_target_slice)

In [92]:
# compute losses of our model
our_train_risks = model.losses(
    hat_train,
    target_train
)

our_val_risks = model.losses(
    hat_val,
    target_val
)

our_test_risks = model.losses(
    hat_test,
    target_test
)

our_dataset_risks = model.losses(
    hat_dataset,
    target_dataset
)

In [93]:
# compute losses of nature sidarthe model
nature_train_risks = model.losses(
    references_train,
    target_train
)

nature_val_risks = model.losses(
    references_val,
    target_val
)

nature_test_risks = model.losses(
    references_test,
    target_test
)

nature_dataset_risks = model.losses(
    references_dataset,
    target_dataset
)

In [94]:
# define utility funcs
def extend_param(value, length):
    len_diff = length - value.shape[0]
    if len_diff > 0:
        return torch.cat((value, value[-1].expand(len_diff)))
    else:
        return value

In [95]:
# get params plot
file_format = ".pdf"
dpi = 144
bbox = 'tight'

max_len = 100

width=None
height=None


base_figures_path = os.path.join(exp_paths, "best")


def filename_from_title(title):
    filename = title.replace("$", "").replace("\\", "").replace(" ","_").replace(".", "")
    return filename + file_format

params_plots = model.plot_params_over_time()
for plot, plot_title in params_plots:
    filename = filename_from_title(plot_title)
    save_path = os.path.join(base_figures_path, filename)
    plot.savefig(save_path, bbox_inches=bbox, transparent=True)


colors = ["b", "g", "r", "c", "m"]
plot_groups = [
    ("Infection Rates", ("alpha", "beta", "gamma", "delta")),
    ("Detection Rates", ("epsilon", "theta")),
    ("Symptoms Developing Rates", ("zeta","eta")),
    ("Threat. Symptoms Develop. Rates", ("mu","nu")),
    ("Fatality Rate", ("tau",)),
    ("Recovery Rates", ("lambda", "kappa", "xi", "rho", "sigma"))
]


for plot_group in plot_groups:
    title, group_keys = plot_group
    #print(group_keys)
    pl_x = list(range(max_len))

    curves = []
    for param_key, color in zip(group_keys, colors):
        param = model.extend_param(model.params[param_key], max_len)
        param_hat_legend = f"$\\{param_key}$"
        param_hat_curve = Curve(pl_x, param.detach().numpy(), '-', param_hat_legend, color) 
        
        param_ref_legend = f"{param_hat_legend} reference"
        param_ref_curve = Curve(pl_x, references[param_key].numpy(), '--', param_ref_legend, color) 
        curves = curves + [param_hat_curve, param_ref_curve]
        
    filename = filename_from_title(title)
    save_path = os.path.join(base_figures_path, filename)
    plot = generic_plot(curves, title, save_path, formatter=format_xtick)


In [96]:
# plot fits

normalized_inferences = model.normalize_values(inferences, model.population)
norm_hat_train = slice_values(normalized_inferences, train_hat_slice)
norm_hat_val = slice_values(normalized_inferences, val_hat_slice)
norm_hat_test = slice_values(normalized_inferences, test_hat_slice)

normalized_targets = model.normalize_values(targets, model.population)
norm_target_train = slice_values(normalized_targets, train_target_slice)
norm_target_val = slice_values(normalized_targets, val_target_slice)
norm_target_test = slice_values(normalized_targets, test_target_slice)


train_range = range(0, train_size)
val_range = range(train_size, train_size + val_size)
test_range = range(train_size + val_size, dataset_size)
dataset_range = range(0, dataset_size)


for key in ["s", "i", "d", "a", "r", "t", "h", "e", "h_detected", "r0"]:
    if key != "r0":
        curr_hat_train = norm_hat_train[key]
        curr_hat_val = norm_hat_val[key]
        curr_hat_test = norm_hat_test[key]
    else:
        curr_hat_train = hat_train[key]
        curr_hat_val = hat_val[key]
        curr_hat_test = hat_test[key]

    if key in normalized_targets:
        target_train = norm_target_train[key]
        target_val = norm_target_val[key]
        target_test = norm_target_test[key]
    else:
        target_train = None
        target_val = None
        target_test = None

    train_curves = model.get_curves(train_range, curr_hat_train, target_train, key, 'r')
    # print(f"{key}: {curr_hat_train.size}")
    val_curves = model.get_curves(val_range, curr_hat_val, target_val, key, 'b')
    # print(f"{key}: {curr_hat_val.size}")
    test_curves = model.get_curves(test_range, curr_hat_test, target_test, key, 'g')
    # print(f"{key}: {curr_hat_test.size}")

    tot_curves = train_curves + val_curves + test_curves

    if references is not None:
        reference_curve = Curve(list(dataset_range), references[key][dataset_target_slice], "--", label="Reference")
        tot_curves = tot_curves + [reference_curve]

    pl_title = f"{key.upper()}"
    filename = filename_from_title(pl_title)
    save_path = os.path.join(base_figures_path, filename)
    fig = generic_plot(tot_curves, pl_title, save_path, formatter=format_xtick)
    pl_title = f"Estimated {key.upper()} on fit"
    

s: 46
s: 20
s: 16
i: 46
i: 20
i: 16
d: 46
d: 20
d: 16
a: 46
a: 20
a: 16
r: 46
r: 20
r: 16
t: 46
t: 20
t: 16
h: 46
h: 20
h: 16
e: 46
e: 20
e: 16
h_detected: 46
h_detected: 20
h_detected: 16
r0: <built-in method size of Tensor object at 0x0000023B846AC368>
r0: <built-in method size of Tensor object at 0x0000023B83171598>
r0: <built-in method size of Tensor object at 0x0000023B8325DD68>
